<a href="https://colab.research.google.com/github/ericsdata/colinsbeer/blob/main/src/WriteReviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 5.2 MB/s 
     |████████████████████████████████| 596 kB 47.5 MB/s 
     |████████████████████████████████| 895 kB 42.8 MB/s 
     |████████████████████████████████| 6.5 MB 34.7 MB/s 
     |████████████████████████████████| 77 kB 6.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
## Packages
import os
import pandas as pd
import torch
import pickle
import numpy as np
import random
import datetime
import time


from torch.utils.data import random_split, RandomSampler, SequentialSampler
from torch.utils.data import Dataset, DataLoader

from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config,  AdamW, get_linear_schedule_with_warmup


In [4]:
## Class to hold beer text
class generative_BD(torch.utils.data.Dataset):
  '''Sequence text tokens
      This means it adds tags to start and end of texts

      Reads in text dataset, & tokenizes

      !!! NEED TO ADD PADDING TOKEN
  '''

  def __init__(self,text_list, tokenizer, text_tags, gpt2_type="gpt2", max_length=768):
    #self.text_list = text_list
    self.tokenizer = tokenizer
    self.input_ids = []
    self.attn_masks = []
    self.text_tags = [text_tags]

    ## In definitoin
    for txt in text_list:

        encodings_dict = tokenizer('<%s>'%(text_tags[0])+ txt + '<%s>'%(text_tags[1]), truncation=True, max_length=max_length, padding="max_length")

        self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
        self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

  def __getitem__(self, idx):
    return self.input_ids[idx], self.attn_masks[idx]

  def __len__(self):
    return len(self.input_ids)

## Func to do some date formatting

def format_time(time_elapsed):
    return str(datetime.timedelta(seconds = int(round(time_elapsed))))


In [5]:
### Data Read

with open("train_data", "rb") as fp:   # Unpickling
   revs = pickle.load(fp)

In [6]:
# Split into training and validation sets
train_size = int(0.8 * len(revs))
val_size = len(revs) - train_size

revs_train, revs_val = random_split(revs, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))


1,907 training samples
  477 validation samples


In [7]:
### Init tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

## Add padding token
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

### Set up the data in datasets
train_dataset = generative_BD(text_list= revs_train, tokenizer = tokenizer, text_tags = ['CLS', 'SEP'])
val_dataset = generative_BD(text_list= revs_val, tokenizer = tokenizer, text_tags = ['CLS', 'SEP'])

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Using pad_token, but it is not set yet.


In [8]:
## Init paramters
batch_size = 2


epochs = 5
learning_rate = 5e-4
warmup_steps = 1e2
epsilon = 1e-8

# this produces sample output every 100 steps
sample_every = 50

In [9]:
# Create the DataLoaders for our training and validation datasets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [10]:
## Init Model

# I'm not really doing anything with the config 
configuration = GPT2Config.from_pretrained('gpt2', output_hidden_states=False)

model = GPT2LMHeadModel.from_pretrained("gpt2", config=configuration)

# this step is necessary because I've added some tokens (bos_token, etc) to the embeddings
# otherwise the tokenizer and model tensors won't match up
model.resize_token_embeddings(len(tokenizer))

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

Embedding(50258, 768)

In [11]:
### SEt model how to run
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Tell pytorch to run this model on the GPU.
device = torch.device("cuda")
model.cuda()

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [12]:
optimizer = AdamW(model.parameters(),
            lr = learning_rate,
            eps = epsilon)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [13]:
### Figure out how many training steps

total_steps = len(revs_train) * epochs

## A scheduler adjusts the learning rate as the training loop progresses

scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = warmup_steps,
                                            num_training_steps = total_steps)

In [14]:
#start training
total_t0 = time.time()
## store records of each run
training_stats = []

### Manual set for each epoch of training
for epoch_i in range(0, epochs):
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    ## Store process starting time
    startT = time.time()
    ## Init loss across training
    total_train_loss = 0
    #begin model training
    model.train()

    for step, batch in enumerate(train_dataset):
        b_input_ids =batch[0].to(device) ## send input ids to model
        b_labels = batch[0].to(device) ## send labels to model (labs are smae as input in gen text)
        b_masks = batch[1].to(device) ## send attention layer to model

        #init gradient at 0
        model.zero_grad()        
        ## feed labels, inputs, and masks to model
        outputs = model(  b_input_ids,
                          labels=b_labels, 
                          attention_mask = b_masks,
                          token_type_ids=None
                        )
        ## Calc loss
        loss = outputs[0]  
        ### loss on batch
        batch_loss = loss.item()
        ## Add it to total training loss
        total_train_loss += batch_loss
        ### Reporting step
        if step % sample_every == 0 and not step == 0:
          elapsed = format_time(time.time() - startT)
          print('  Batch {:>5,}  of  {:>5,}. Loss: {:>5,}.   Elapsed: {:}.'.format(step, len(train_dataset), batch_loss, elapsed))

          model.eval()
          ### Output some samples so you know its working
          sample_outputs = model.generate(
                                bos_token_id = random.randint(1,30000)
                                ,do_sample = True
                                ,top_k = 50
                                ,max_lenght = 200
                                ,top_p = 0.095
                                ,num_return_sequences = 1
          )

          for i, sample_output in enumerate(sample_outputs):
                  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

          model.train()

        loss.backward()

        optimizer.step()

        scheduler.step()
    ## Calcu average loss over all epochs
    avg_train_loss = total_train_loss / len(train_dataset)

    ## Ouput how long epoch took
    training_time = format_time(time.time() - startT)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))

     # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    t0 = time.time()

    model.eval()

    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)
        
        with torch.no_grad():        

            outputs  = model(b_input_ids, 
#                            token_type_ids=None, 
                             attention_mask = b_masks,
                            labels=b_labels)
          
            loss = outputs[0]  
            
        batch_loss = loss.item()
        total_eval_loss += batch_loss        

    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    validation_time = format_time(time.time() - t0)    

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 5 ========
Training...


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  Batch    50  of  1,907. Loss: 0.4144846200942993.   Elapsed: 0:00:28.
0:  bipartisan and


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  Batch   100  of  1,907. Loss: 0.763882040977478.   Elapsed: 0:00:56.
0:  increasing the, and the amount of


...

  Batch 1,800  of  1,907. Loss: 0.1861092448234558.   Elapsed: 0:16:34.
0: ulCLS>42935 1 Bottle.  Pours a clear golden color with a small white


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  Batch 1,850  of  1,907. Loss: 0.20013566315174103.   Elapsed: 0:17:01.
0:  slidingCLS>42935 1 Bottle.  Pours a clear golden color with a small white


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  Batch 1,900  of  1,907. Loss: 0.06851405650377274.   Elapsed: 0:17:29.
0:  DellCLS>46471 1 Bottle. Golden with a small white head. Aroma is ho

  Average training loss: 0.20
  Training epoch took: 0:17:33

Running Validation...
  Validation Loss: 0.41
  Validation took: 0:01:18

Training complete!
Total training took 1:34:11 (h:mm:ss)


In [15]:
### Prompty Tokens
'''
beerID  name                     
42935	   Trader Joes Hofbrau Bock	
43176	  Trader Joes Vienna Style Lager	
46471	  Ridgeway IPA	
65888	  Saranac Brown Ale	
117319	Harpoon 100 Barrel Series #30 - Island Creek Oyster Stout	
2519	  Shipyard Chamberlain Pale Ale	
29028	  De Hemel Nieuw Ligt Grand Cru	
3202	  Penn Dark Lager Beer	
4302	  Gearys London Porter	
56242	  East End Gratitude	

Then 1 for good or 0 for bad
'''

beer = input('Enter a beer ID: ')
rating = input('Review Good or Bad [1/0] ? ')



Enter a beer ID: 42935
Review Good or Bad [1/0] ? 1


In [21]:
model.eval()

prompt_good = beer + " 1" + rating
prompt_bad = beer + " 1" + rating

generated = torch.tensor(tokenizer.encode(prompt_good)).unsqueeze(0)
generated = generated.to(device)

#print(generated)

sample_outputs = model.generate(
                                generated, 
                                #bos_token_id=random.randint(1,30000),
                                do_sample=True,   
                                top_k=50, 
                                max_length = 300,
                                top_p=0.95, 
                                num_return_sequences=3
                                )
print('\nGood reviews for the beer are:\n')
for i, sample_output in enumerate(sample_outputs):
  print("{}: {}\n\n".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

generated = torch.tensor(tokenizer.encode(prompt_bad)).unsqueeze(0)
generated = generated.to(device)

#print(generated)

sample_outputs = model.generate(
                                generated, 
                                #bos_token_id=random.randint(1,30000),
                                do_sample=True,   
                                top_k=50, 
                                max_length = 300,
                                top_p=0.95, 
                                num_return_sequences=3
                                )
print('Bad reviews for the beer are:\n')
for i, sample_output in enumerate(sample_outputs):
  print("{}: {}\n\n".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Good reviews for the beer are:

0: 42935 11.2 oz bottle. Aroma is sweet. Taste is sweet. A little sticky in the end. Well made stuff.<SEP>,.<SEPours3.42902904345610S 0290S>11S>43SS>43S>1129010S>3 (4S3. 34. Wow290290 (1SEPours290S2901143SEPours2903290 -311S>56S>3290S>56S-4S>10S>56S> 4PoursS>11290 (SEPoursS>25343S	Sells	6511


1: 42935 11.5 fl oz bottle.  Aroma is sweet, and slightly alcoholic.  Flavor is sweet, slightly alcoholic, a little bit chalky, and slightly bitter.<SEP>8a.	S=S -3431143 -44293SEP	4 a.290811:464 rottenSEPoursS11S.4645S>3SEP	. SE	464S>4SEPoursS<SEPours29046411. 10311290113290 (4430SEPoursSEP	SEP	3023	SEPou3SEP>3.11


2: 42935 11.2 fl oz bottle, courtesy of my friend Olga. Pours a rich, lightly hazy, dark-brown color with a white head. The aroma is sweet and floral. The taste is sweet and floral. Medium mouthfeel and finish. Pretty standard mouthfeel. Just ok.<SEP>6571 1  04329065290 rotten rotten29010S>464S>464 (S44242 (71.10SEPours2903<SEP	290 (46429011SEP>11290S>4

In [17]:
# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

output_dir = './model_save/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)



Saving model to ./model_save/


('./model_save/tokenizer_config.json',
 './model_save/special_tokens_map.json',
 './model_save/vocab.json',
 './model_save/merges.txt',
 './model_save/added_tokens.json')

In [19]:
!zip -r /content/model_save.zip /content/model_save

  adding: content/model_save/ (stored 0%)
  adding: content/model_save/tokenizer_config.json (deflated 65%)
  adding: content/model_save/pytorch_model.bin (deflated 9%)
  adding: content/model_save/merges.txt (deflated 53%)
  adding: content/model_save/config.json (deflated 51%)
  adding: content/model_save/vocab.json (deflated 63%)
  adding: content/model_save/special_tokens_map.json (deflated 69%)
  adding: content/model_save/added_tokens.json (stored 0%)


In [20]:
from google.colab import files
files.download("/content/model_save.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>